# Grid Search
Run separately for each LSTM-backed model one-by-one

In [ ]:
# Load Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

import warnings

warnings.filterwarnings("ignore")

from typing import List

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasRegressor
from keras.constraints import MaxNorm

import keras

import os, sys
rootpath = ".."
sys.path.insert(0, f"{os.getcwd()}/{rootpath}/base_models")
import model_prep

step_back = 6  # window size = 6*5 = 30 mins

In [ ]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
       'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
       'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
    target = 'EnergyConsumption'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, _ = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model(dropout_rate = 0.0, weight_constraint = 2.0, lstmcells = 32, activation = 'tanh', optimizer = "Adamax"):
        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        model.compile(loss='mse', optimizer=optimizer)
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, loss="mse", verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    print(grid_result)

In [ ]:
batch_size = [32, 64, 72, 128] # selecting 32
epochs = [50, 100, 200]  # selecting 200
param_grid = dict(batch_size=batch_size, epochs=epochs)
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
# neurons = [6, 16, 32, 64, 128]
# lstmcells = [1, 6, 16, 32, 64]
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)
# param_grid = dict(model__activation=activation)

In [ ]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
       'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
       'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
    target = 'EnergyConsumption'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, _ = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model(dropout_rate = 0.0, weight_constraint = 2.0, lstmcells = 32, activation = 'tanh'):
        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, loss="mse", batch_size=32, epochs=200, verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    print(grid_result)

In [ ]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax'] # selecting Adam
param_grid = dict(optimizer=optimizer)
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

In [ ]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
       'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
       'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
    target = 'EnergyConsumption'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, _ = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model(activation, dropout_rate = 0.0, weight_constraint = 2.0, lstmcells = 32, optimizer = "Adam"):
        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        model.compile(loss='mse', optimizer=optimizer)
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, loss="mse", verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    print(grid_result)

In [ ]:
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'] # selecting relu
param_grid = dict(model__activation=activation)
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

In [ ]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
       'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
       'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
    target = 'EnergyConsumption'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, _ = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model(lstmcells, activation="relu", dropout_rate = 0.0, weight_constraint = 2.0, optimizer = "Adam"):
        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        model.compile(loss='mse', optimizer=optimizer)
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, loss="mse", verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    print(grid_result)

In [ ]:
param_grid = dict(model__lstmcells=[16, 32, 64, 128]) # selecting 64
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

In [ ]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
       'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
       'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
    target = 'EnergyConsumption'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, _ = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model(dropout_rate, weight_constraint, lstmcells=64, activation="relu", optimizer = "Adam"):
        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        model.compile(loss='mse', optimizer=optimizer)
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, loss="mse", verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    print(grid_result)

In [ ]:
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0] # select 0.0
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # select 4.0
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

In [ ]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
       'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
       'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
    target = 'EnergyConsumption'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, _ = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
        step_back=6
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model(step_back, dropout_rate=0.0, weight_constraint=4.0, lstmcells=64, activation="relu", optimizer = "Adam"):
        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        model.compile(loss='mse', optimizer=optimizer)
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, loss="mse", verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    print(grid_result)

In [ ]:
param_grid = dict(model__step_back=[1, 3, 6, 12, 24]) # selecting step_back = 6

# Call the run_grid_search function with the modified param_grid
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

```
batch size: 32
epochs: 200
optimizer: Adam
dropout rate: 0.0
weight constraints: 4.0
step_back: 6
lstm cells: 64
activation: relu
```